In [1]:
import joblib
import pandas as pd
import numpy as np

# Load artifacts
rf = joblib.load("rf_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoders = joblib.load("label_encoders.pkl")
feature_names = joblib.load("feature_names.pkl")  # <-- THE KEY FIX

def predict_churn(input_dict):

    # Turn dictionary into dataframe
    df = pd.DataFrame([input_dict])

    # -----------------------------
    # APPLY LABEL ENCODERS IF NEEDED
    # -----------------------------
    for col, le in label_encoders.items():
        if col in df.columns:
            df[col] = le.transform(df[col].astype(str))

    # -----------------------------
    # ADD MISSING COLUMNS
    # -----------------------------
    for col in feature_names:
        if col not in df.columns:
            df[col] = 0

    # -----------------------------
    # ENSURE CORRECT COLUMN ORDER
    # -----------------------------
    df = df[feature_names]

    # -----------------------------
    # SCALE + PREDICT
    # -----------------------------
    df_scaled = scaler.transform(df)

    proba = rf.predict_proba(df_scaled)[0][1]
    pred = int(proba > 0.5)

    return pred, proba


# Example input
sample = {
    'Age': 35,
    'Gender': 1,
    'Tenure': 12,
    'Usage Frequency': 10,
    'Support Calls': 3,
    'Payment Delay': 2,
    'Total Spend': 500,
    'Last Interaction': 30,
    'Subscription Type_Basic': 1,
    'Subscription Type_Standard': 0,
    'Subscription Type_Premium': 0,
    'Contract Length_Annual': 1,
    'Contract Length_Monthly': 0,
    'Contract Length_Quarterly': 0,
    'Avg_Monthly_Spend': 41.6,
    'Support_Intensity': 0.3,
    'Recency_Tenure_Ratio': 2.5,
}

print(predict_churn(sample))

(1, np.float64(0.9083875521523662))
